In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import json
import pandas as pd
import datasets

from tqdm.auto import tqdm
from glob import glob

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("C-LLM/train/wang_cscd_train.json", "r", encoding='utf-8') as f:
	wang_cscd_train = json.load(f)
with open("C-LLM/train/cscd_train.json", "r", encoding='utf-8') as f:
	cscd_train = json.load(f)
with open("C-LLM/train/cscd_dev.json", "r", encoding='utf-8') as f:
	cscd_dev = json.load(f)

In [7]:
sentences_noisy = []
sentences = []
categories = []

for row in wang_cscd_train:
    sentence_noisy, sentence, category = row['src'], row['tgt'], 'wang_cscd'
    if len(sentence)!=len(sentence_noisy):
        raise ValueError('sentence length not matched')
    sentences_noisy.append(sentence_noisy)
    sentences.append(sentence)
    categories.append(category)

for row in cscd_train:
    sentence_noisy, sentence, category = row['src'], row['tgt'], 'cscd'
    if len(sentence)!=len(sentence_noisy):
        raise ValueError('sentence length not matched')
    sentences_noisy.append(sentence_noisy)
    sentences.append(sentence)
    categories.append(category)

In [8]:
train_df = pd.DataFrame({
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
    'category': categories
})  
train_ds = datasets.Dataset.from_pandas(train_df)

In [9]:
sentences_noisy = []
sentences = []
categories = []

for row in cscd_dev:
    sentence_noisy, sentence, category = row['src'], row['tgt'], 'cscd'
    if len(sentence)!=len(sentence_noisy):
        raise ValueError('sentence length not matched')
    sentences_noisy.append(sentence_noisy)
    sentences.append(sentence)
    categories.append(category)

In [10]:
dev_df = pd.DataFrame({
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
    'category': categories
})  
dev_ds = datasets.Dataset.from_pandas(dev_df)

In [4]:
sentences_noisy = []
sentences = []
categories = []

for path in glob('C-LLM/test/*.json'):
    filename = os.path.basename(path)
    category = filename.split('.json')[0]
    with open(path, "r", encoding='utf-8') as f:
        datas = json.load(f)
    for row in datas:
        sentence_noisy, sentence = row['src'], row['tgt']
        if len(sentence)!=len(sentence_noisy):
            raise ValueError('sentence length not matched')
        sentences_noisy.append(sentence_noisy)
        sentences.append(sentence)
        categories.append(category)

In [5]:
test_df = pd.DataFrame({
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
    'category': categories
})  
test_ds = datasets.Dataset.from_pandas(test_df)

In [6]:
test_df

,sentence,sentence_noisy,category
0,"在最近的福特资本市场日投资者介绍会上,首席产品平台和运营官","在最近的福特资本市场日投资者介绍会上,首席产品平台和运营干",car
1,"前脸大尺寸镀铬进气栅搭配上竖条状LED日行灯,让该车看上去十分有气势。","前脸大尺寸镀格进气栅搭配上竖条状LED日行灯,让该车看上去十分有气势。",car
2,"汽车在运行过程中,不可避免地会产生一定的积碳和磨损,貌似都是小问题,时间久了,就会造成燃油浪...","汽车在运行过程中,不可避免地会产生一定的积碳和磨损,貌似都是小问题,时间久了,就会造成燃油浪...",car
3,"在这之前,宏光MINI","再这之前,宏光MINI",car
4,"这是社会责任,不要说什么扣分罚钱,就算扣分罚钱你也应该让。","这是社会责任,不要说什么扣分罚钱,就算扣分发钱你也应该让。",car
...,...,...,...
26726,"因受家庭浓厚的文化熏陶,少玉6岁时就进了度尾铸德小学读书,以后又先后就读于陶德初中和国立福州...","因受家庭浓厚的文化熏陶,少玉6岁时就进了度尾铸德小学读书,以后又先后就读于陶德初中和国立福州...",enc
26727,"用于合奏钢琴的调律,还可以起到统一标准的作用。","用于合奏钢琴的调律,还可以起到统一标准的作用。",enc
26728,"竹邑人薛综至孙子薛兼三代任太子少傅,“由综至兼,世传东宫,谈者美之。”","竹邑人薛综至孙子薛兼三代任太子少傅,“由综至兼,世传东宫,谈者美之。”",enc
26729,"9年未发行专辑的那英于近期发行全新大碟《那又怎样》,同时还将顺势启动全新主题和概念的“那英·...","9年未发行专辑的那英于近期发行全新大碟《那又怎样》,同时还将顺势启动全新主题和概念的“那英·...",enc


In [11]:
ds = datasets.DatasetDict({
    'train': train_ds,
    'dev': dev_ds,
    'test': test_ds
})

In [12]:
ds.push_to_hub(repo_id=f'jwengr/C-LLM', private=False)

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.56s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/jwengr/C-LLM/commit/8e29fbcb7f4c49e1bd457984e05e37c1048f36f1', commit_message='Upload dataset', commit_description='', oid='8e29fbcb7f4c49e1bd457984e05e37c1048f36f1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jwengr/C-LLM', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jwengr/C-LLM'), pr_revision=None, pr_num=None)